In [18]:
import psycopg2
import folium
import json

In [19]:
def show_layer_map(layer_id, dbname, user, password, host, port):
    """
    Recupera il GeoJSON dalla tabella geom_data per un dato layerid
    e visualizza la geometria su una mappa Folium.
    """

    # Connessione al DB
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cursor = conn.cursor()

    try:
        # Query per recuperare il GeoJSON associato al layerid
        sql = """
        SELECT geojson
        FROM geom_data
        WHERE layerid = %s
        """
        cursor.execute(sql, (layer_id,))
        result = cursor.fetchone()

        if not result:
            print(f"Nessun record trovato per layerid = {layer_id}")
            return

        # Il GeoJSON è in formato testo/jsonb: lo convertiamo in dizionario Python
        geojson_str = result[0]  # se è jsonb, in psycopg2 potrebbe già essere un dict
        if isinstance(geojson_str, dict):
            geojson_data = geojson_str
        else:
            # se è stringa, facciamo il parsing
            geojson_data = json.loads(geojson_str)

        # Estraiamo una coordinata (lon, lat) per centrare la mappa
        # Assumiamo che la geometria sia un Feature con chiave 'geometry'
        # e che sia un MultiPolygon o Polygon
        geometry = geojson_data['geometry']
        # Prendiamo la prima coordinata della prima "shell" del poligono
        coords = geometry['coordinates'][0][0][0]
        # NB: se Polygon e non MultiPolygon, adattare l'indice

        # In GeoJSON l'ordine è [lon, lat]
        center_lon, center_lat = coords[0], coords[1]

        # Creiamo la mappa
        m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

        # Aggiungiamo il layer GeoJSON
        folium.GeoJson(
            geojson_data,
            name=f"Layer {layer_id}"
        ).add_to(m)

        folium.LayerControl().add_to(m)

        # Mostriamo la mappa
        # 1) Se siamo in un Jupyter Notebook:
        return m

        # 2) Se vogliamo salvare in HTML:
        # m.save(f"mappa_{layer_id}.html")
        # print(f"Mappa salvata come mappa_{layer_id}.html")

    finally:
        cursor.close()
        conn.close()

In [ ]:
mappa = show_layer_map("layer_id", "dbname", "user", "password", "host", port)
mappa